# Introduction

This notebook demonstrates basic usage of BioThings Explorer, an engine for autonomously querying a distributed knowledge graph. BioThings Explorer can answer two classes of queries -- "PREDICT" and "EXPLAIN".  PREDICT queries are described in [PREDICT_demo.ipynb](PREDICT_demo.ipynb). Here, we describe EXPLAIN queries and how to use BioThings Explorer to execute them.  A more detailed overview of the BioThings Explorer systems is provided in [these slides](https://docs.google.com/presentation/d/1QWQqqQhPD_pzKryh6Wijm4YQswv8pAjleVORCPyJyDE/edit?usp=sharing).

EXPLAIN queries are designed to **identify plausible reasoning chains to explain the relationship between two entities**.  For example, in this notebook, we explore the question:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"*Why does imatinib have an effect on the treatment of chronic myelogenous leukemia (CML)?*"  

Later, we also compare those results to a similar query looking at imatinib's role in treating gastrointestinal stromal tumors (GIST).

**To experiment with an executable version of this notebook, [load it in Google Colaboratory](https://colab.research.google.com/github/biothings/biothings_explorer/blob/master/jupyter%20notebooks/EXPLAIN_demo.ipynb).**

## Step 0: Load BioThings Explorer modules

First, install the `biothings_explorer` and `biothings_schema` packages, as described in this [README](https://github.com/biothings/biothings_explorer/blob/master/jupyter%20notebooks/README.md#prerequisite).  This only needs to be done once (but including it here for compability with [colab](https://colab.research.google.com/)).

In [1]:
!pip install git+https://github.com/biothings/biothings_explorer#egg=biothings_explorer
!pip install git+https://github.com/biothings/biothings_schema.py#egg=biothings_schema

Next, import the relevant modules:

* **Hint**: Find corresponding bio-entity representation used in BioThings Explorer based on user input (could be any database IDs, symbols, names)
* **FindConnection**: Find intermediate bio-entities which connects user specified input and output

In [2]:
# import modules from biothings_explorer
from biothings_explorer.hint import Hint
from biothings_explorer.user_query_dispatcher import FindConnection

## Step 1: Find representation of "chronic myelogenous leukemia" and "imatinib" in BTE

In this step, BioThings Explorer translates our query strings "chronic myelogenous leukemia" and "imatinib" into BioThings objects, which contain mappings to many common identifiers.  Generally, the top result returned by the `Hint` module will be the correct item, but you should confirm that using the identifiers shown.

Search terms can correspond to any child of [BiologicalEntity](https://biolink.github.io/biolink-model/docs/BiologicalEntity.html) from the [Biolink Model](https://biolink.github.io/biolink-model/docs/), including `DiseaseOrPhenotypicFeature` (e.g., "lupus"), `ChemicalSubstance` (e.g., "acetaminophen"), `Gene` (e.g., "CDK2"), `BiologicalProcess` (e.g., "T cell differentiation"), and `Pathway` (e.g., "Citric acid cycle").

In [3]:
ht = Hint()
# find all potential representations of CML
cml_hint = ht.query("chronic myelogenous leukemia")
# select the correct representation of CML
cml = cml_hint['DiseaseOrPhenotypicFeature'][0]
cml

{'mondo': 'MONDO:0011996',
 'doid': 'DOID:8552',
 'umls': 'C1292772',
 'mesh': 'D015464',
 'name': 'chronic myelogenous leukemia',
 'display': 'mondo(MONDO:0011996) doid(DOID:8552) umls(C1292772) mesh(D015464) name(chronic myelogenous leukemia) ',
 'type': 'DiseaseOrPhenotypicFeature',
 'primary': {'identifier': 'mondo',
  'cls': 'DiseaseOrPhenotypicFeature',
  'value': 'MONDO:0011996'}}

In [4]:
# find all potential representations of imatinib
imatinib_hint = ht.query("imatinib")
# select the correct representation of imatinib
imatinib = imatinib_hint['ChemicalSubstance'][0]
imatinib

{'chembl': 'CHEMBL941',
 'drugbank': 'DB00619',
 'name': 'IMATINIB',
 'pubchem': 5291,
 'umls': 'C0935989',
 'mesh': 'D000068877',
 'display': 'chembl(CHEMBL941) drugbank(DB00619) name(IMATINIB) pubchem(5291) umls(C0935989) mesh(D000068877) ',
 'type': 'ChemicalSubstance',
 'primary': {'identifier': 'chembl',
  'cls': 'ChemicalSubstance',
  'value': 'CHEMBL941'}}

## Step 2: Find intermediate nodes connecting imatinib and chronic myelogenous leukemia

In this section, we find all paths in the knowledge graph that connect imatinib and chronic myelogenous leukemia.  To do that, we will use `FindConnection`.  This class is a convenient wrapper around two advanced functions for **query path planning** and **query path execution**. More advanced features for both query path planning and query path execution are in development and will be documented in the coming months. 

The parameters for `FindConnection` are described below:


In [5]:
help(FindConnection.__init__)

Help on function __init__ in module biothings_explorer.user_query_dispatcher:

__init__(self, input_obj, output_obj, intermediate_nodes, registry=None)
    Find relationships in the Knowledge Graph between an Input Object and an Output Object.
    
    params
    ------
    input_obj (required): must be an object returned from Hint corresponding to a specific biomedical entity.
                          Examples: 
            Hint().query("Fanconi anemia")['DiseaseOrPhenotypicFeature'][0]
            Hint().query("acetaminophen")['ChemicalSubstance'][0]
    
    output_obj (required): must EITHER be an object returned from Hint corresponding to a specific biomedical
                           entity, OR be a string or list of strings corresponding to Biolink Entity classes.
                           Examples:
            Hint().query("acetaminophen")['ChemicalSubstance'][0]
            'Gene'
            ['Gene','ChemicalSubstance']
    
    intermediate_nodes (required): the semantic

Here, we formulate a `FindConnection` query with "CML" as the `input_ojb`, "imatinib" as the `output_obj`.  We further specify with the `intermediate_nodes` parameter that we are looking for paths joining chronic myelogenous leukemia and imatinib with *one* intermediate node that is a Gene.  (The ability to search for longer reasoning paths that include additional intermediate nodes will be added shortly.)

In [6]:
fc = FindConnection(input_obj=cml, output_obj=imatinib, intermediate_nodes='Gene')

We next execute the `connect` method, which performs the **query path planning** and **query path execution** process.  In short, BioThings Explorer is deconstructing the query into individual API calls, executing those API calls, then assembling the results.

A verbose log of this process is displayed below:

In [7]:
# set verbose=True will display all steps which BTE takes to find the connection
fc.connect(verbose=True)

========== QUERY PARAMETER SUMMARY ==========

BTE will find paths that join 'chronic myelogenous leukemia' and 'IMATINIB'. Paths will have 1 intermediate node.

Intermediate node #1 will have these type constraints: Gene


========== QUERY #1 -- fetch all Gene entities linked to 'chronic myelogenous leukemia' ==========

==== Step #1: Query path planning ====

Because chronic myelogenous leukemia is of type 'DiseaseOrPhenotypicFeature', BTE will query our meta-KG for APIs that can take 'DiseaseOrPhenotypicFeature' as input

BTE found 3 apis:

API 1. semmeddisease(1 API call)
API 2. biolink_disease2gene(1 API call)
API 3. mydisease.info(1 API call)


==== Step #2: Query path execution ====
NOTE: API requests are dispatched in parallel, so the list of APIs below is ordered by query time.

API 2.1: http://pending.biothings.io/semmed/query (POST "q=C0023473,C1292772&scopes=umls&fields=ASSOCIATED_WITH.gene,AFFECTS.gene,AFFECTS.protein,CAUSES_reverse.gene,AFFECTS_reverse.protein,AFFECTS_rev

## Step 3: Display and Filter results
This section demonstrates post-query filtering done in Python. Later, more advanced filtering functions will be added to the **query path execution** module for interleaved filtering, thereby enabling longer query paths. More details to come...

First, all matching paths can be exported to a data frame. Let's examine a sample of those results.

In [8]:
df = fc.display_table_view()

# because UMLS is not currently well-integrated in our ID-to-object translation system, removing UMLS-only entries here
patternDel = "^umls:C\d+"
filter = df.node1_id.str.contains(patternDel)
df = df[~filter]

df.sample(10)

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,node1_id,node1_name,node1_type,pred2,pred2_source,pred2_api,pred2_pubmed,output_id,output_name,output_type
84,chronic myelogenous leukemia,DiseaseOrPhenotypicFeature,associatedWith,biolink,biolink_disease2gene,None,entrez:613,BCR,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,chembl:CHEMBL941,IMATINIB,ChemicalSubstance
135,chronic myelogenous leukemia,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:51014,TMED7,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,chembl:CHEMBL941,IMATINIB,ChemicalSubstance
124,chronic myelogenous leukemia,DiseaseOrPhenotypicFeature,associatedWith,semmed,semmeddisease,,entrez:3717,JAK2,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,chembl:CHEMBL941,IMATINIB,ChemicalSubstance
26,chronic myelogenous leukemia,DiseaseOrPhenotypicFeature,associatedWith,semmed,semmeddisease,,entrez:640,BLK,Gene,target,drugcentral,mychem.info,None,chembl:CHEMBL941,IMATINIB,ChemicalSubstance
83,chronic myelogenous leukemia,DiseaseOrPhenotypicFeature,associatedWith,biolink,biolink_disease2gene,None,entrez:613,BCR,Gene,target,dgidb,dgidb_chemical2gene,None,chembl:CHEMBL941,IMATINIB,ChemicalSubstance
121,chronic myelogenous leukemia,DiseaseOrPhenotypicFeature,associatedWith,semmed,semmeddisease,,entrez:3439,IFNA1,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,chembl:CHEMBL941,IMATINIB,ChemicalSubstance
33,chronic myelogenous leukemia,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:2268,FGR,Gene,target,drugcentral,mychem.info,None,chembl:CHEMBL941,IMATINIB,ChemicalSubstance
119,chronic myelogenous leukemia,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:2260,FGFR1,Gene,target,drugcentral,mychem.info,None,chembl:CHEMBL941,IMATINIB,ChemicalSubstance
50,chronic myelogenous leukemia,DiseaseOrPhenotypicFeature,associatedWith,semmed,semmeddisease,,entrez:7422,VEGFA,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,chembl:CHEMBL941,IMATINIB,ChemicalSubstance
112,chronic myelogenous leukemia,DiseaseOrPhenotypicFeature,associatedWith,semmed,semmeddisease,,entrez:1630,DCC,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,chembl:CHEMBL941,IMATINIB,ChemicalSubstance


While most results are based on edges from [semmed](https://skr3.nlm.nih.gov/SemMed/), edges from [DGIdb](http://www.dgidb.org/), [biolink](https://monarchinitiative.org/), [disgenet](http://www.disgenet.org/), [mydisease.info](https://mydisease.info) and [drugcentral](http://drugcentral.org/) were also retrieved from their respective APIs.  

Next, let's look to see which genes are mentioned the most.

In [9]:
df.node1_name.value_counts().head(10)

ABL1      20
BCR       15
KIT       10
ABCB1      8
STAT5A     4
BIRC5      4
AKT1       4
JAK2       3
PSMD9      2
CXCR4      2
Name: node1_name, dtype: int64

Not surprisingly, the top two genes that BioThings Explorer found that join imatinib to CML are *ABL1* and *BCR*, the two genes that are fused in the "Philadelphia chromosome", the genetic abnormality that underlies CML, *and* the validate target of imatinib.

Let's examine some of the PubMed articles linking **CML to *ABL1*** and ***ABL1* to imatinib**.

In [10]:
articles = fc.display_edge_info('chronic myelogenous leukemia', 'ABL1')[0]['info']['bts:pubmed']
print("There are "+str(len(articles))+" articles supporting the edge between CML and ABL1. Sampling of 10 of those:")
x = [print("http://pubmed.gov/"+str(x)) for x in articles[0:10] ]

There are 63 articles supporting the edge between CML and ABL1. Sampling of 10 of those:
http://pubmed.gov/12623848
http://pubmed.gov/20537386
http://pubmed.gov/23676790
http://pubmed.gov/11964322
http://pubmed.gov/22870928
http://pubmed.gov/11423618
http://pubmed.gov/20010464
http://pubmed.gov/24109527
http://pubmed.gov/21562040
http://pubmed.gov/20557306


In [11]:
articles = fc.display_edge_info('ABL1', 'IMATINIB')[4]['info']['bts:publication']
print("There are "+str(len(articles))+" articles supporting the edge between ABL1 and imatinib. Sampling of 10 of those:")
x = [print("http://pubmed.gov/"+str(x)) for x in articles[0:10] ]

There are 16 articles supporting the edge between ABL1 and imatinib. Sampling of 10 of those:
http://pubmed.gov/11287973
http://pubmed.gov/20537386
http://pubmed.gov/26758680
http://pubmed.gov/20525993
http://pubmed.gov/23502220
http://pubmed.gov/19075254
http://pubmed.gov/15930265
http://pubmed.gov/17496201
http://pubmed.gov/12637609
http://pubmed.gov/11287972


## Comparing results between CML and GIST

Let's perform another BioThings Explorer query, this time looking to EXPLAIN the relationship between imatinib and gastrointestinal stromal tumors (GIST), another disease treated by imatinib.

In [12]:
ht = Hint()
# find all potential representations of CML
gist_hint = ht.query("gastrointestinal stromal tumor")
# select the correct representation of CML
gist = gist_hint['DiseaseOrPhenotypicFeature'][0]
gist

{'mondo': 'MONDO:0011719',
 'doid': 'DOID:9253',
 'umls': 'C3179349',
 'mesh': 'D046152',
 'name': 'gastrointestinal stromal tumor',
 'display': 'mondo(MONDO:0011719) doid(DOID:9253) umls(C3179349) mesh(D046152) name(gastrointestinal stromal tumor) ',
 'type': 'DiseaseOrPhenotypicFeature',
 'primary': {'identifier': 'mondo',
  'cls': 'DiseaseOrPhenotypicFeature',
  'value': 'MONDO:0011719'}}

In [13]:
fc = FindConnection(input_obj=gist, output_obj=imatinib, intermediate_nodes='Gene')

In [14]:
fc.connect(verbose=False) # skipping the verbose log here


After id-to-object translation, BTE retrieved 31 unique objects.

After id-to-object translation, BTE retrieved 245 unique objects.


In [15]:
df = fc.display_table_view()

# because UMLS is not currently well-integrated in our ID-to-object translation system, removing UMLS-only entries here
patternDel = "^umls:C\d+"
filter = df.node1_id.str.contains(patternDel)
df = df[~filter]

df.sample(10)

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,node1_id,node1_name,node1_type,pred2,pred2_source,pred2_api,pred2_pubmed,output_id,output_name,output_type
77,gastrointestinal stromal tumor,DiseaseOrPhenotypicFeature,associatedWith,semmed,semmeddisease,,entrez:5156,PDGFRA,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,chembl:CHEMBL941,IMATINIB,ChemicalSubstance
19,gastrointestinal stromal tumor,DiseaseOrPhenotypicFeature,associatedWith,disgenet,mydisease.info,None,entrez:3815,KIT,Gene,target,drugcentral,mychem.info,None,chembl:CHEMBL941,IMATINIB,ChemicalSubstance
50,gastrointestinal stromal tumor,DiseaseOrPhenotypicFeature,associatedWith,biolink,biolink_disease2gene,,entrez:5156,PDGFRA,Gene,target,drugbank,mychem.info,"15894928,15921304,15928335,15946589,15995325",chembl:CHEMBL941,IMATINIB,ChemicalSubstance
72,gastrointestinal stromal tumor,DiseaseOrPhenotypicFeature,associatedWith,semmed,semmeddisease,,entrez:5156,PDGFRA,Gene,target,drugcentral,mychem.info,None,chembl:CHEMBL941,IMATINIB,ChemicalSubstance
14,gastrointestinal stromal tumor,DiseaseOrPhenotypicFeature,associatedWith,disgenet,mydisease.info,None,entrez:3815,KIT,Gene,target,drugcentral,mychem.info,None,chembl:CHEMBL941,IMATINIB,ChemicalSubstance
15,gastrointestinal stromal tumor,DiseaseOrPhenotypicFeature,associatedWith,disgenet,mydisease.info,None,entrez:3815,KIT,Gene,target,drugbank,mychem.info,"16865565,16087693,17458563,17369583,17559139",chembl:CHEMBL941,IMATINIB,ChemicalSubstance
28,gastrointestinal stromal tumor,DiseaseOrPhenotypicFeature,associatedWith,semmed,semmeddisease,,entrez:3815,KIT,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,chembl:CHEMBL941,IMATINIB,ChemicalSubstance
49,gastrointestinal stromal tumor,DiseaseOrPhenotypicFeature,associatedWith,biolink,biolink_disease2gene,,entrez:5156,PDGFRA,Gene,target,drugcentral,mychem.info,None,chembl:CHEMBL941,IMATINIB,ChemicalSubstance
85,gastrointestinal stromal tumor,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:5156,PDGFRA,Gene,target,drugcentral,mychem.info,None,chembl:CHEMBL941,IMATINIB,ChemicalSubstance
16,gastrointestinal stromal tumor,DiseaseOrPhenotypicFeature,associatedWith,disgenet,mydisease.info,None,entrez:3815,KIT,Gene,target,dgidb,dgidb_chemical2gene,None,chembl:CHEMBL941,IMATINIB,ChemicalSubstance


In [16]:
df.node1_name.value_counts().head(10)

PDGFRA    42
KIT       25
BRAF       9
CD34       4
PDGFA      2
BIRC5      2
SNRPE      1
NF1        1
VEGFA      1
NCOR1      1
Name: node1_name, dtype: int64

Here, the top two genes that BioThings Explorer found that join imatinib to GIST are *PDGFRA* and *KIT*, the most commonly mutated genes found in GIST *and* validated targets of imatinib.

While several of the listed genes would be considered positive controls, others on the list could be viewed as **testable hypotheses and discovery opportunities** to be evaluated by domain experts.

## Conclusions and caveats

This notebook demonstrated the use of BioThings Explorer in EXPLAIN mode to investigate the relationship between imatinib and two diseases that it treats -- chronic myelogenous leukemia (CML) and gastrointestinal stromal tumors (GIST).  In each case, BioThings Explorer autonomously queried a **distributed knowledge graph of biomedical APIs** to find the most common genes, and in each case the relevant targets were retrieved.

There are still many areas for improvement (and some areas in which BioThings Explorer is still buggy).  And of course, BioThings Explorer is dependent on the accessibility of the APIs that comprise the distributed knowledge graph.  Nevertheless, we encourage users to try other variants of the EXPLAIN queries demonstrated in this notebook.